In [9]:
import base64
import os
import ipywidgets as wg
from IPython.display import display

In [10]:
from dotenv import load_dotenv
load_dotenv()

True

In [11]:
url = os.environ['HDA_URL']
user = os.environ['HDA_USER']
password = os.environ['HDA_PASSWORD']

In [12]:
parameters = 'url: '+url+'\nuser: '+user+'\npassword: '+password

In [13]:
with open(os.path.join(os.environ['HOME'],'.hdarc'), 'w') as fp:
    fp.write(parameters)

In [14]:
from hda import Client

In [27]:
%%capture
#Avoid printing secrets
%%capture
c = Client()

In [ ]:
# Import necessary libraries for the project

# Data
import pandas as pd

# Spatial data
# import geopandas as gpd
import rasterio as rio

# Machine learning

# Data visualization and widgets
import contextily as ctx
import matplotlib.pyplot as plt

# Requests (API's)
import requests
import json

In [28]:
query = {
  "datasetId": "EO:ESA:DAT:SENTINEL-3:OL_2_LFR___",
  "boundingBoxValues": [
    {
      "name": "bbox",
      "bbox": [
        -7.499583333333334,
        42.3020216796485,
        11.999333333333333,
        54.75429195536845
      ]
    }
  ],
  "dateRangeSelectValues": [
    {
      "name": "position",
      "start": "2021-01-01T00:00:00.000Z",
      "end": "2021-01-02T00:00:00.000Z"
    }
  ],
  "stringChoiceValues": [
    {
      "name": "productType",
      "value": "LFR"
    },
    {
      "name": "processingLevel",
      "value": "LEVEL2"
    }
  ]
}

# The following line runs the query
matches = c.search(query)

# The following line prints the products returned by the query
print(matches)

#The download starts. All the products found in the query are downloaded consecutively
matches.download()

2022-09-21 15:45:20,945 DEBUG ===> GET https://wekeo-broker.apps.mercator.dpi.wekeo.eu/databroker/gettoken
2022-09-21 15:45:20,951 DEBUG Starting new HTTPS connection (1): wekeo-broker.apps.mercator.dpi.wekeo.eu:443
2022-09-21 15:45:21,820 DEBUG https://wekeo-broker.apps.mercator.dpi.wekeo.eu:443 "GET /databroker/gettoken HTTP/1.1" 200 61
2022-09-21 15:45:21,821 DEBUG <=== {"access_token": "f8838272-53ea-30b2-a270-612cae3b88e7"}
2022-09-21 15:45:21,822 DEBUG Token is f8838272-53ea-30b2-a270-612cae3b88e7
2022-09-21 15:45:21,822 DEBUG ===> POST https://wekeo-broker.apps.mercator.dpi.wekeo.eu/databroker/datarequest
2022-09-21 15:45:21,823 DEBUG ===> POST {"datasetId": "EO:ESA:DAT:SENTINEL-3:OL_2_LFR___", "boundingBoxValues": [{"na...
2022-09-21 15:45:22,165 DEBUG https://wekeo-broker.apps.mercator.dpi.wekeo.eu:443 "POST /databroker/datarequest HTTP/1.1" 200 93
2022-09-21 15:45:22,166 DEBUG <=== {"jobId": "V6bVT9nBTGSEtY8ZRCLuD6U8bPc", "status": "started", "results": [], ...
2022-09-21 15:

SearchResults[items=6,volume=448.6M,jobId=V6bVT9nBTGSEtY8ZRCLuD6U8bPc]


2022-09-21 15:45:25,453 DEBUG https://wekeo-broker.apps.mercator.dpi.wekeo.eu:443 "POST /databroker/dataorder HTTP/1.1" 200 80
2022-09-21 15:45:25,454 DEBUG <=== {"orderId": "WZk6wiexSm-DWQttbA5RCNOuAcQ", "status": "running", "message": null}
2022-09-21 15:45:25,454 DEBUG Sleeping 1 seconds
2022-09-21 15:45:26,456 DEBUG ===> GET https://wekeo-broker.apps.mercator.dpi.wekeo.eu/databroker/dataorder/status/WZk6wiexSm-DWQttbA5RCNOuAcQ
2022-09-21 15:45:26,755 DEBUG https://wekeo-broker.apps.mercator.dpi.wekeo.eu:443 "GET /databroker/dataorder/status/WZk6wiexSm-DWQttbA5RCNOuAcQ HTTP/1.1" 200 353
2022-09-21 15:45:26,757 DEBUG <=== {"status": "completed", "message": "Done!", "downloadUri": "c8387e34-29f3-51e...
2022-09-21 15:45:26,759 INFO Downloading https://wekeo-broker.apps.mercator.dpi.wekeo.eu/databroker/dataorder/download/WZk6wiexSm-DWQttbA5RCNOuAcQ to S3B_OL_2_LFR____20210101T112211_20210101T112511_20210102T153133_0179_047_251_2160_LN1_O_NT_002.SEN3.zip (88.1M)
2022-09-21 15:45:27,388 D

In [ ]:
import snappy